# import library

In [ ]:
import matplotlib.pyplot
import numpy as np
import os
from PIL import Image
from collections import defaultdict
from itertools import product
from sklearn.model_selection import train_test_split
import shutil
import re
import glob
from scipy import ndimage
import pickle
from six.moves import cPickle as pickle
from six.moves import range
#from __future__ import division, print_function, absolute_import
import tflearn
from tflearn.data_utils import shuffle
from tflearn.layers.core import input_data, dropout, fully_connected
from tflearn.layers.conv import conv_2d, max_pool_2d
from tflearn.layers.estimator import regression
from tflearn.data_preprocessing import ImagePreprocessing
from tflearn.data_augmentation import ImageAugmentation
import tensorflow as tf

C:\anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
C:\anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
C:\anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
C:\anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:519: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is 

curses is not supported on this machine (please install/reinstall curses for an optimal experience)








In [ ]:
%cd "D:\\Ananconda\\login detect\\dataset"

D:\Ananconda\login detect\dataset


# Load Data

In [ ]:
# khai báo width height, rotation, shift, scale parameter

width = 32
height = 32

posshiftshift_min = -5
posshiftshift_max = 5
scales = [0.9, 1.1]
rot_min = -15
rot_max = 15

dir = 'dataset'
imgdir = os.path.join(dir, 'main_dataset')
pp_dir = os.path.join(
    dir, 'dataset8')
annot = 'result.txt'

In [ ]:
annot_train = np.loadtxt(os.path.join(dir, annot), dtype='a')
print('train_annotation: %d, %d ' % (annot_train.shape))

train_annotation: 4536, 7 


# Crop và augmente data

## augmente data

In [ ]:

# lấy bounding box từ text file

def parse_annot(annot):
    fn = annot[0].decode('utf-8')
    class_name = annot[1].decode('utf-8')
    train_subset_class = annot[2].decode('utf-8')
    return fn, class_name, train_subset_class

def get_rect(annot):             
    rect = defaultdict(int)
    x1, y1, x2, y2 = rect_coord(annot[3:])
    cx, cy, wid, hgt = center_wid_hgt(x1, y1, x2, y2)
    rect['x1'] = x1
    rect['y1'] = y1
    rect['x2'] = x2
    rect['y2'] = y2
    rect['cx'] = cx
    rect['cy'] = cy
    rect['wid'] = wid
    rect['hgt'] = hgt
    return rect

In [ ]:
# shift vị trí bounding box trong hình

def posshift(annot, im):   
    posshift_ims = []
    posshift_suffixes = []

    rect = get_rect(annot)
    for sx, sy in product(                        
            range(posshiftshift_min, posshiftshift_max),
            range(posshiftshift_min, posshiftshift_max)):
        cx = rect['cx'] + sx
        cy = rect['cy'] + sy
        cropped_im = im.crop((cx - rect['wid'] // 2, cy - rect['hgt'] // 2,
                              cx + rect['wid'] // 2, cy + rect['hgt'] // 2))
        resized_im = cropped_im.resize((width, height))
        posshift_ims.append(resized_im)
        posshift_suffixes.append('p' + str(sx) + str(sy))
        cropped_im.close()

    return posshift_ims, posshift_suffixes

In [ ]:
# sclae bounding box theo parameter

def scale(annot, im):   
    scale_ims = []
    scale_suffixes = []

    rect = get_rect(annot)
    for s in scales:
        w = int(rect['wid'] * s)
        h = int(rect['hgt'] * s)
        cropped_im = im.crop((rect['cx'] - w // 2, rect['cy'] - h // 2,
                              rect['cx'] + w // 2, rect['cy'] + h // 2))
        resized_im = cropped_im.resize((width, height))
        scale_ims.append(resized_im)
        scale_suffixes.append('s' + str(s))
        cropped_im.close()

    return scale_ims, scale_suffixes

In [ ]:
# rotate bounding box theo parameter


def rotate(annot, im):   
    rotate_ims = []
    rotate_suffixes = []

    rect = get_rect(annot)
    for r in range(rot_min, rot_max):
        rotated_im = im.rotate(r)
        cropped_im = rotated_im.crop(
            (rect['cx'] - rect['wid'] // 2, rect['cy'] - rect['hgt'] // 2,
             rect['cx'] + rect['wid'] // 2, rect['cy'] + rect['hgt'] // 2))
        resized_im = cropped_im.resize((width, height))
        rotate_ims.append(resized_im)
        rotate_suffixes.append('r' + str(r))
        rotated_im.close()
        cropped_im.close()

    return rotate_ims, rotate_suffixes

## crop hình

In [ ]:
#Cropping the logo

def crop(annot, im):                        
    x1, y1, x2, y2 = rect_coord(annot[3:])
    cropped_im = im.crop((x1, y1, x2, y2))
    cropped_im = cropped_im.resize((width, height))
    cropped_suffix = 'p00'
    return [cropped_im], [cropped_suffix]


def rect_coord(annot_part):
    return list(map(int, annot_part))   


def center_wid_hgt(x1, y1, x2, y2):
    cx = x1 + (x2 - x1) // 2
    cy = y1 + (y2 - y1) // 2
    wid = (x2 - x1)
    hgt = (y2 - y1)
    return cx, cy, wid, hgt

In [ ]:
# kiểm tra điều kiện bỏ logo, save hình và đóng file


def is_skip(annot_part):
    x1, y1, x2, y2 = rect_coord(annot_part)
    _, _, wid, hgt = center_wid_hgt(x1, y1, x2, y2)
    if wid <= 0 or hgt <= 0:
        return True
    else:
        return False

def save_im(annot, cnt, *args):         
    fn, class_name, train_subset_class = parse_annot(annot)
    dst_dir = os.path.join(pp_dir, class_name)
    if not os.path.exists(dst_dir):
        os.makedirs(dst_dir)
    for i, arg in enumerate(args):
        for im, suffix in zip(arg[0], arg[1]):
            save_fn = '_'.join([
                fn.split('.')[0], class_name, train_subset_class, str(cnt),
                suffix
            ]) + os.path.splitext(fn)[1]
            im.save(os.path.join(dst_dir, save_fn))


def close_im(*args):
    for ims in args:
        for im in ims:
            im.close()

In [ ]:
# parent function để gọi tất cà các sub functions

def crop_and_aug(annot_train):    
    cnt_per_file = defaultdict(int)
    for annot in annot_train:
        # for generating a file name
        fn, _, _ = parse_annot(annot)
        cnt_per_file[fn] += 1

        # skip if width or height equal zero
        if is_skip(annot[3:]):
            print('Skip: ', fn)
            continue

        # open an image
        im = Image.open(os.path.join(imgdir, fn))

        # normal cropping
        cropped_ims, cropped_suffixes = crop(annot, im)

        # augment by shifting a center
        shifted_ims, shifted_suffixes = posshift(annot, im)

        # augment by scaling
        scaled_ims, scaled_suffixes = scale(annot, im)

        # augment by rotation
        rotated_ims, rotated_suffixes = rotate(annot, im)

        # save images
        save_im(annot, cnt_per_file[fn], [cropped_ims, cropped_suffixes],
                [shifted_ims, shifted_suffixes], [scaled_ims, scaled_suffixes],
                [rotated_ims, rotated_suffixes])

        # close image file
        close_im([im], cropped_ims, shifted_ims, scaled_ims, rotated_ims)

## tạo dataset

In [ ]:
# gọi tất cà các function và tạo dataset mới

def crop_and_aug_with_none(annot_train, with_none=False):
    # root directory to save processed images
    if not os.path.exists(pp_dir):
        os.makedirs(pp_dir)

    # crop images and apply augmentation
    crop_and_aug(annot_train)

    # print results
    org_imgs = [img for img in os.listdir(imgdir)]
    crop_and_aug_imgs = [
        fname
        for root, dirs, files in os.walk(pp_dir)
        for fname in glob.glob(os.path.join(root, '*.jpg'))  # look for the file with .jpg extension.
    ]
    print('original: %d' % (len(org_imgs)))
    print('cropped: %d' % (len(crop_and_aug_imgs)))

# splitting data

In [ ]:
# tạo train và test set
def do_train_test_split():                                   
    class_names = [cls for cls in os.listdir(pp_dir)]
   # create directories under a particular class name.
    for class_name in class_names:                           
        if os.path.exists(                                  
                os.path.join(pp_dir, class_name, 'train')):
            continue
        if os.path.exists(
                os.path.join(pp_dir, class_name, 'test')):
            continue

        imgs = [
            img
            for img in os.listdir(
                os.path.join(pp_dir, class_name))
        ]
        # train=0.75, test=0.25
        train_imgs, test_imgs = train_test_split(imgs)
        # move images to train or test directory
        # create directories
        os.makedirs(os.path.join(pp_dir, class_name, 'train'))         
        os.makedirs(os.path.join(pp_dir, class_name, 'test'))
        for img in train_imgs:
            dst = os.path.join(pp_dir, class_name, 'train')
            src = os.path.join(pp_dir, class_name, img)
            # moving image into that directory
            shutil.move(src, dst)                                      
        for img in test_imgs:
            dst = os.path.join(pp_dir, class_name, 'test')
            src = os.path.join(pp_dir, class_name, img)
            shutil.move(src, dst)

# gọi function

In [ ]:
crop_and_aug_with_none(annot_train)

Skip:  2662264721.jpg
Skip:  2662264721.jpg
Skip:  2662264721.jpg
Skip:  2662264721.jpg
Skip:  2662264721.jpg
original: 1079
cropped: 598092


In [ ]:
do_train_test_split()

# Tạo Pickle File chứa dataset

## tạo thông số cho pickle file

In [ ]:
#Parameters của hình
width = 32     
height = 32    
channel = 3         
pix_val = 255.0

dir = 'dataset'
# Directory where processed images are stored
pp_dir = os.path.join(dir, 'dataset8') 
# tên pickle file
pickle_file = 'logo_dataset.pickle'    
# sớ hình sử dụng cho training và validate
train_size = 70000  
val_size = 5000
# sớ hình sử dụng cho test
test_size = 7000

In [ ]:
# tạo một array chứa height, width và channel các hình trong dataset

def array(nb_rows, image_width, image_height, image_ch=1):     
    if nb_rows:
        dataset = np.ndarray(                               #  stores its height, width and channel into an array
            (nb_rows, image_height, image_width, image_ch), dtype=np.float32)
        labels = np.ndarray(nb_rows, dtype=np.int32)        #  stores its labels
    else:
        dataset, labels = None, None
    return dataset, labels

## các function cho việc tạo pickle

In [ ]:
# dụng nhập pickle files của 10 classes vào chung pickle file.

def combine(pickle_files, train_size, val_size=0):     
    num_classes = len(pickle_files)
    valid_dataset, valid_labels = array(val_size, width,
                                              height, channel)
    train_dataset, train_labels = array(train_size, width,
                                              height, channel)
    vsize_per_class = val_size // num_classes
    tsize_per_class = train_size // num_classes

    start_v, start_t = 0, 0
    end_v, end_t = vsize_per_class, tsize_per_class
    end_l = vsize_per_class + tsize_per_class
    for label, pickle_file in enumerate(pickle_files):
        try:
            with open(pickle_file, 'rb') as f:
                logo_set = pickle.load(f)
                np.random.shuffle(logo_set)
                if valid_dataset is not None:
                    valid_logo = logo_set[:vsize_per_class, :, :, :]
                    valid_dataset[start_v:end_v, :, :, :] = valid_logo
                    valid_labels[start_v:end_v] = label
                    start_v += vsize_per_class
                    end_v += vsize_per_class
                train_logo = logo_set[vsize_per_class:end_l, :, :, :]
                train_dataset[start_t:end_t, :, :, :] = train_logo
                train_labels[start_t:end_t] = label
                start_t += tsize_per_class
                end_t += tsize_per_class
        except Exception as e:
            print('Unable to process data from', pickle_file, ':', e)
            raise
    return valid_dataset, valid_labels, train_dataset, train_labels

In [ ]:
# tạo pickle files cho một class

def makepickle(train_dataset, train_labels, valid_dataset, valid_labels,   
                test_dataset, test_labels):
    try:
        f = open(pickle_file, 'wb')
        save = {
            'train_dataset': train_dataset,
            'train_labels': train_labels,
            'valid_dataset': valid_dataset,
            'valid_labels': valid_labels,
            'test_dataset': test_dataset,
            'test_labels': test_labels,
        }
        pickle.dump(save, f, pickle.HIGHEST_PROTOCOL)      # Saving data of the images into a pickle file
        f.close()
    except Exception as e:
        print('Unable to save data to', pickle_file, ':', e)
        raise

In [ ]:
# load hình logo từ thông số

def load_logo(data_dir):    
    image_files = os.listdir(data_dir)      
    dataset = np.ndarray(
        shape=(len(image_files), height, width, channel),
        dtype=np.float32)
    print(data_dir)
    num_images = 0
    for image in image_files:
        image_file = os.path.join(data_dir, image)
        try:
            image_data = (matplotlib.pyplot.imread(image_file).astype(float) -        
                          pix_val / 2) / pix_val
            if image_data.shape != (height, width, channel):
                raise Exception('Unexpected image shape: %s' %
                                str(image_data.shape))
            dataset[num_images, :, :] = image_data
            num_images = num_images + 1
        except IOError as e:
            print('Could not read:', image_file, ':', e,
                  '-it\'s ok, skipping.')

    dataset = dataset[0:num_images, :, :]                           
    print('Full dataset tensor:', dataset.shape)       # Tell processed number of images for a particular class 
    print('Mean:', np.mean(dataset))                   # Calculate mean over that entire class
    print('Standard deviation:', np.std(dataset))      # Calculate standard deviation over that entire class
    return dataset

In [ ]:
# tạo  parent function Pickle file từ các sub functions
def pickling(data_dirs, force=False):     
    dataset_names = []
    for dir in data_dirs:
        set_filename = dir + '.pickle'
        dataset_names.append(set_filename)

         
        if os.path.exists(set_filename) and force:    
    
            print('%s already present - Skipping pickling. ' % set_filename)
        else:
            print('Pickling %s.' % set_filename)
            dataset = load_logo(dir)
            try:
                with open(set_filename, 'wb') as f:
                    pickle.dump(dataset, f, pickle.HIGHEST_PROTOCOL)
            except Exception as e:
                print('Unable to save data to', set_filename, ':', e)
    return dataset_names

In [ ]:
def randomize(dataset, labels):
    permutation = np.random.permutation(labels.shape[0])
    shuffled_dataset = dataset[permutation, :, :]
    shuffled_labels = labels[permutation]
    return shuffled_dataset, shuffled_labels

# tạo pickle file

In [ ]:
CLASS_NAME = [
   'Apple', 'BMW','Heineken','HP','Intel','Mini','Starbucks','Vodafone', 'unknown', 'Ferrari'
]

In [ ]:
dirs = [
        os.path.join(pp_dir, class_name, 'train')      # Look into all the train folder of the class
        for class_name in CLASS_NAME
    ]
test_dirs = [
        os.path.join(pp_dir, class_name, 'test')        # Look into all the test folder of the class
        for class_name in CLASS_NAME
    ]

train_datasets = pickling(dirs)
test_datasets = pickling(test_dirs)

Pickling flickrData\processedF2\Apple\train.pickle.
flickrData\processedF2\Apple\train
Full dataset tensor: (25443, 32, 32, 3)
Mean: 0.11680751
Standard deviation: 0.29468784
Pickling flickrData\processedF2\BMW\train.pickle.
flickrData\processedF2\BMW\train
Full dataset tensor: (11781, 32, 32, 3)
Mean: -0.07805122
Standard deviation: 0.30858785
Pickling flickrData\processedF2\Heineken\train.pickle.
flickrData\processedF2\Heineken\train
Full dataset tensor: (15939, 32, 32, 3)
Mean: -0.08372974
Standard deviation: 0.26778436
Pickling flickrData\processedF2\HP\train.pickle.
flickrData\processedF2\HP\train
Full dataset tensor: (10989, 32, 32, 3)
Mean: -0.11454076
Standard deviation: 0.33904904
Pickling flickrData\processedF2\Intel\train.pickle.
flickrData\processedF2\Intel\train
Full dataset tensor: (12177, 32, 32, 3)
Mean: 0.1226464
Standard deviation: 0.32122466
Pickling flickrData\processedF2\Mini\train.pickle.
flickrData\processedF2\Mini\train
Full dataset tensor: (10395, 32, 32, 3)
Me

In [ ]:
valid_dataset, valid_labels, train_dataset, train_labels = combine(train_datasets, train_size, val_size)# function called for merging

In [ ]:
a,b,test_dataset, test_labels= combine(test_datasets, test_size, val_size=0)

train_dataset, train_labels = randomize(train_dataset, train_labels)   # function called for randomizing
valid_dataset, valid_labels = randomize(valid_dataset, valid_labels)  
test_dataset, test_labels = randomize(test_dataset, test_labels)

In [ ]:
makepickle(train_dataset, train_labels, valid_dataset, valid_labels,test_dataset, test_labels)# function called for making a pickle file.
statinfo = os.stat(pickle_file)                         # Shows size of the file 
print('Compressed pickle size:', statinfo.st_size)

Compressed pickle size: 1007944511


# test the dataset

In [ ]:
def read_data():
    with open("logo_dataset.pickle", 'rb') as f:
        save = pickle.load(f)
        X = save['train_dataset']       # assign X as train dataset
        Y = save['train_labels']        # assign Y as train labels 
        X_test = save['test_dataset']   # assign X_test as test dataset
        Y_test = save['test_labels']    #assign Y_test as test labels
        del save
    return [X, X_test], [Y, Y_test]

def reformat(dataset, labels):   
    dataset = dataset.reshape((-1, 32, 32,3)).astype(np.float32)    # Reformatting shape array to give a scalar value for dataset.  
    labels = (np.arange(10) == labels[:, None]).astype(np.float32) 
    return dataset, labels

dataset, labels = read_data()
X,Y = reformat(dataset[0], labels[0])
X_test, Y_test = reformat(dataset[1], labels[1])
print('Training set', X.shape, Y.shape)
print('Test set', X_test.shape, Y_test.shape)            

# Shuffle the data
X, Y = shuffle(X, Y)    # Imported from TFLearn.

Training set (70000, 32, 32, 3) (70000, 10)
Test set (7000, 32, 32, 3) (7000, 10)
